#### Step 1: Setup and Load Data


In [1]:
import pandas as pd
import numpy as np
import sys
import os

# Add project directory to path
sys.path.append(os.path.abspath(".."))

from database.db_connection import DatabaseConnector

db_connector = DatabaseConnector()
engine = db_connector.get_engine()

query_forecast = "SELECT * FROM historical_forecast;"
query_weather = "SELECT * FROM historical_weather;"

forecast_df = pd.read_sql(query_forecast, engine)
weather_df = pd.read_sql(query_weather, engine)

print("Data Loaded Successfully!")

Data Loaded Successfully!


#### Step 2: Data Overview

In [2]:
print("=== Forecast Data Overview ===")
print(forecast_df.info())
print("\nSample Data (Forecast):")
print(forecast_df.head())

print("\n=== Weather Data Overview ===")
print(weather_df.info())
print("\nSample Data (Weather):")
print(weather_df.head())

=== Forecast Data Overview ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8784 entries, 0 to 8783
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   id                      8784 non-null   int64         
 1   datetime                8784 non-null   datetime64[ns]
 2   temperature_2m          8784 non-null   float64       
 3   relative_humidity_2m    8784 non-null   float64       
 4   precipitation           8784 non-null   float64       
 5   rain                    8784 non-null   float64       
 6   snowfall                8784 non-null   float64       
 7   snow_depth              8784 non-null   float64       
 8   weather_code            8784 non-null   float64       
 9   pressure_msl            8784 non-null   float64       
 10  surface_pressure        8784 non-null   float64       
 11  cloud_cover             8784 non-null   float64       
 12  cloud_cover_low  

The dataset contains **historical weather data** and **historical forecast data** for **Zurich** from **January 1, 2024, to December 31, 2024**, recorded **hourly**. Both datasets include **8784 entries**.  

### **Table Overview: Columns and Data**  

- **Datetime**: Timestamps for each hourly observation.  
- **Temperature and Humidity**:  
  - **temperature_2m** – Air temperature at 2 meters (°C).  
  - **relative_humidity_2m** – Relative humidity (%).  
- **Precipitation and Snow**:  
  - **precipitation**, **rain** – Total precipitation and rainfall (mm).  
  - **snowfall**, **snow_depth** – Snowfall and snow depth (mm).  
- **Pressure and Cloud Cover**:  
  - **pressure_msl**, **surface_pressure** – Atmospheric pressure at mean sea level and surface (hPa).  
  - **cloud_cover**, **cloud_cover_low/mid/high** – Cloud coverage (%).  
- **Wind Data**:  
  - **wind_speed_10m**, **wind_direction_10m**, **wind_gusts_10m** – Wind speed, direction (°), and gusts (m/s).  
- **Soil Conditions**:  
  - **soil_temperature** – Soil temperature at surface or shallow depth (°C).  
  - **soil_moisture** – Soil moisture content.  
- **Categories**:  
  - **temp_category**, **wind_category**, **rain_category** – Categorical classifications for temperature, wind, and rain.  

This structured data provides detailed weather and forecast variables for time-series analysis and comparisons.

#### Step 3: Missing Values

In [3]:
print("\n=== Missing Values in Forecast Data ===")
print(forecast_df.isnull().sum())

print("\n=== Missing Values in Weather Data ===")
print(weather_df.isnull().sum())


=== Missing Values in Forecast Data ===
id                        0
datetime                  0
temperature_2m            0
relative_humidity_2m      0
precipitation             0
rain                      0
snowfall                  0
snow_depth                0
weather_code              0
pressure_msl              0
surface_pressure          0
cloud_cover               0
cloud_cover_low           0
cloud_cover_mid           0
cloud_cover_high          0
visibility                0
wind_speed_10m            0
wind_direction_10m        0
wind_gusts_10m            0
soil_temperature_0cm      0
soil_moisture_0_to_1cm    0
temp_category             0
wind_category             0
rain_category             0
dtype: int64

=== Missing Values in Weather Data ===
id                             0
datetime                       0
temperature_2m                 0
relative_humidity_2m           0
precipitation                  0
rain                           0
snowfall                       0
sno

#### Observation Step 3: Missing Values 

- **Forecast Data:** No missing values. Dataset is **complete**.  
- **Weather Data:** Only **`snow_depth`** has **575 missing values** (~6.5%).  

**Note:**  
This is **not an issue with our processing**. Manual API testing confirmed these values are **missing directly from the source API**.

#### Step 4: Descriptive Statistics

In [4]:
print("\n=== Descriptive Statistics (Forecast) ===")
print(forecast_df.describe())

print("\n=== Descriptive Statistics (Weather) ===")
print(weather_df.describe())


=== Descriptive Statistics (Forecast) ===
                id             datetime  temperature_2m  relative_humidity_2m  \
count  8784.000000                 8784     8784.000000           8784.000000   
mean   4392.500000  2024-07-01 23:30:00       11.978711             76.951958   
min       1.000000  2024-01-01 00:00:00       -6.700000             21.000000   
25%    2196.750000  2024-04-01 11:45:00        6.200000             68.000000   
50%    4392.500000  2024-07-01 23:30:00       11.400000             81.000000   
75%    6588.250000  2024-10-01 11:15:00       17.700000             89.000000   
max    8784.000000  2024-12-31 23:00:00       33.900000            100.000000   
std    2535.866716                  NaN        7.749807             16.674198   

       precipitation         rain     snowfall   snow_depth  weather_code  \
count    8784.000000  8784.000000  8784.000000  8784.000000   8784.000000   
mean        0.147199     0.133959     0.003004     0.001503     19.008766

#### **Observations Step 4: Descriptive Statistics**  

#### **Forecast Data**  
- **Temperature:** Ranges from **-6.7°C** to **33.9°C** (mean: **12.0°C**).  
- **Humidity:** High average (**77%**), max at **100%**, indicating frequent moisture.  
- **Precipitation/Rain:** Mostly **dry** (median = **0.0 mm**), but peaks at **17 mm**.  
- **Snowfall:** Rare, with max at **2.1 cm**.  
- **Pressure:** Stable, avg **1017 hPa**.  
- **Wind Speed:** Avg **5.4 m/s**, max gusts at **69.8 m/s**.  
- **Soil Temp:** Avg **12.5°C**, indicating moderate ground warmth.  

#### **Weather Data**  
- **Temperature:** Slightly cooler than forecasts (mean: **11.2°C**, min: **-9.5°C**).  
- **Humidity:** Higher avg (**82.5%**) than forecasts, more moisture observed.  
- **Precipitation/Rain:** Lower max (**8.8 mm**) vs forecast (**17 mm**).  
- **Snowfall:** Higher max (**3.43 cm**), suggesting potential overforecasting.  
- **Wind Speed:** Slightly stronger winds (mean: **6.3 m/s**, gusts up to **77 m/s**).  
- **Soil Temp:** Cooler avg (**11.2°C**) than forecast (**12.5°C**).  

**Conclusion:**  
Weather observations show **slightly cooler temperatures**, **lower rainfall**, and **higher humidity** compared to forecasts. Wind speeds and gusts were **higher than predicted**.

#### Step 5: Correlation Analysis (Numerical Data)

In [5]:
# Exclude non-numeric columns for correlation analysis
forecast_numeric = forecast_df.select_dtypes(include=[np.number])
weather_numeric = weather_df.select_dtypes(include=[np.number])

print("\n=== Correlation Analysis (Forecast) ===")
print(forecast_numeric.corr())

print("\n=== Correlation Analysis (Weather) ===")
print(weather_numeric.corr())


=== Correlation Analysis (Forecast) ===
                              id  temperature_2m  relative_humidity_2m  \
id                      1.000000        0.076836              0.152281   
temperature_2m          0.076836        1.000000             -0.578767   
relative_humidity_2m    0.152281       -0.578767              1.000000   
precipitation          -0.008196       -0.010281              0.134066   
rain                   -0.005366        0.006472              0.125349   
snowfall                0.014468       -0.084534              0.052186   
snow_depth              0.096026       -0.164283              0.024712   
weather_code           -0.008445       -0.194694              0.358925   
pressure_msl            0.222697       -0.323596              0.299786   
surface_pressure        0.242818       -0.184780              0.224967   
cloud_cover            -0.027185       -0.198994              0.225981   
cloud_cover_low         0.108438       -0.389019              0.462366 

### **Observations Step 5: Correlation Analysis**  

#### **Forecast Data**  
- **Temperature vs Humidity:** Strong **negative correlation (-0.58)** — higher temperatures are linked to lower humidity.  
- **Precipitation vs Rain:** Extremely **high correlation (0.98)** — confirms rain is the main contributor to precipitation.  
- **Snowfall vs Snow Depth:** Moderate **correlation (0.23)** — snow depth depends on snowfall but also other factors (melting, accumulation).  
- **Cloud Cover vs Precipitation:** **Positive correlation (0.14–0.25)** — higher cloud cover often brings precipitation.  
- **Soil Temperature vs Air Temperature:** **Very strong correlation (0.97)** — expected due to close physical interaction.  
- **Soil Moisture vs Precipitation:** **Moderate correlation (0.34)** — rainfall impacts soil moisture but other factors like evaporation and drainage also play a role.  
- **Wind Speed vs Wind Gusts:** **High correlation (0.89)** — gusts are naturally related to wind speed patterns.  

#### **Weather Data**  
- **Temperature vs Humidity:** Similar **negative correlation (-0.50)** — consistent with forecasts.  
- **Rain vs Precipitation:** **Very strong correlation (0.96)** — validates measurement consistency.  
- **Snowfall vs Snow Depth:** **Positive correlation (0.14)** — slightly weaker than forecast, possibly due to different snow accumulation or melting rates.  
- **Cloud Cover vs Precipitation:** Moderate **correlation (0.18–0.36)** — cloudy conditions often accompany rain but not always.  
- **Soil Temperature vs Air Temperature:** **Very strong correlation (0.97)** — matches forecasts, emphasizing soil-air temperature dependency.  
- **Soil Moisture vs Precipitation:** **Moderate correlation (0.33)** — reflects rainfall's effect on soil moisture but with external influences.  

### **Conclusion:**  
The correlations align with physical expectations (e.g., temperature-humidity trade-offs, rain-precipitation links). Forecast and weather data show consistent patterns, validating forecast accuracy for key variables like temperature, rainfall, and wind.

#### Step 6: Categorical Analysis

In [6]:
# Frequency of temperature categories
print("\n=== Temperature Categories (Forecast) ===")
print(forecast_df['temp_category'].value_counts())

print("\n=== Temperature Categories (Weather) ===")
print(weather_df['temp_category'].value_counts())

# Frequency of wind categories
print("\n=== Wind Categories (Forecast) ===")
print(forecast_df['wind_category'].value_counts())

print("\n=== Wind Categories (Weather) ===")
print(weather_df['wind_category'].value_counts())

# Frequency of rain categories
print("\n=== Rain Categories (Forecast) ===")
print(forecast_df['rain_category'].value_counts())

print("\n=== Rain Categories (Weather) ===")
print(weather_df['rain_category'].value_counts())



=== Temperature Categories (Forecast) ===
temp_category
mild    5268
heiß    3039
kalt     477
Name: count, dtype: int64

=== Temperature Categories (Weather) ===
temp_category
mild    5460
heiß    2796
kalt     528
Name: count, dtype: int64

=== Wind Categories (Forecast) ===
wind_category
schwach    7739
mäßig      1045
Name: count, dtype: int64

=== Wind Categories (Weather) ===
wind_category
schwach    7328
mäßig      1453
stark         3
Name: count, dtype: int64

=== Rain Categories (Forecast) ===
rain_category
trocken      8181
nass          581
sehr nass      22
Name: count, dtype: int64

=== Rain Categories (Weather) ===
rain_category
trocken      8028
nass          741
sehr nass      15
Name: count, dtype: int64


### **Observations Step 6: Categorical Analysis**  

#### **Temperature Categories**  
- **Forecast:**  
  - **Mild (60%)** is the dominant category, followed by **heiß (35%)** and **kalt (5%)**.  
- **Weather:**  
  - Similar pattern with **mild (62%)**, **heiß (32%)**, and **kalt (6%)**.  
- **Insight:** Forecast and weather data agree well, showing most days are mild or warm, with few cold periods.  

---

#### **Wind Categories**  
- **Forecast:**  
  - **Schwach (88%)** dominates, with **mäßig (12%)**.  
- **Weather:**  
  - Mostly **schwach (83%)**, some **mäßig (16%)**, and **stark (<1%)**.  
- **Insight:** Both datasets predict mostly weak winds. Observed data has a few strong wind events not captured in forecasts.  

---

#### **Rain Categories**  
- **Forecast:**  
  - Mostly **trocken (93%)**, with **nass (6.6%)** and rare **sehr nass (0.25%)**.  
- **Weather:**  
  - Similar distribution: **trocken (91%)**, **nass (8.4%)**, and **sehr nass (0.17%)**.  
- **Insight:** Both datasets agree that most days are dry, with occasional light rain and very few heavy rain events.  

---

### **Conclusion:**  
- Forecast and weather data are **highly consistent** across all categories.  
- Minor deviations in wind and rain categories may reflect **local variations or measurement differences**.  
- The data indicates **mild temperatures, weak winds, and predominantly dry conditions** for the observed period.

#### Step 7: Time-Based Analysis

In [7]:
print("\n=== Time Analysis: Forecast Data ===")
print("Start Date:", forecast_df['datetime'].min())
print("End Date:", forecast_df['datetime'].max())
print("Time Frequency (Forecast):")
print(forecast_df['datetime'].dt.date.value_counts().sort_index())

print("\n=== Time Analysis: Weather Data ===")
print("Start Date:", weather_df['datetime'].min())
print("End Date:", weather_df['datetime'].max())
print("Time Frequency (Weather):")
print(weather_df['datetime'].dt.date.value_counts().sort_index())


=== Time Analysis: Forecast Data ===
Start Date: 2024-01-01 00:00:00
End Date: 2024-12-31 23:00:00
Time Frequency (Forecast):
datetime
2024-01-01    24
2024-01-02    24
2024-01-03    24
2024-01-04    24
2024-01-05    24
              ..
2024-12-27    24
2024-12-28    24
2024-12-29    24
2024-12-30    24
2024-12-31    24
Name: count, Length: 366, dtype: int64

=== Time Analysis: Weather Data ===
Start Date: 2024-01-01 00:00:00
End Date: 2024-12-31 23:00:00
Time Frequency (Weather):
datetime
2024-01-01    24
2024-01-02    24
2024-01-03    24
2024-01-04    24
2024-01-05    24
              ..
2024-12-27    24
2024-12-28    24
2024-12-29    24
2024-12-30    24
2024-12-31    24
Name: count, Length: 366, dtype: int64


### **Observations Step 7: Time Analysis**  

#### **Forecast Data**  
- **Start Date:** January 1, 2024, 00:00:00  
- **End Date:** December 31, 2024, 23:00:00  
- **Frequency:** Data is recorded **hourly (24 entries per day)** across **366 days** (leap year).  

---

#### **Weather Data**  
- **Start Date:** January 1, 2024, 00:00:00  
- **End Date:** December 31, 2024, 23:00:00  
- **Frequency:** Same hourly records (**24 entries per day**) covering **366 days**.  

---

### **Insights:**  
- Both datasets have **complete and consistent time coverage** for the entire year, including a leap year (366 days).  
- The **hourly granularity** provides **high-resolution temporal data** for detailed weather analysis.  
- There are **no missing timestamps**, ensuring **data continuity** for time-series modeling or trend analysis.